In [22]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext


In [23]:
conf = SparkConf().setAppName("preprocess_data")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=preprocess_data, master=local[*]) created by __init__ at <ipython-input-2-9c3e9459808c>:2 

In [24]:
train_data = sqlContext.read.json("reviews_training_set.json")

In [25]:
train_rdd = train_data.select("user_id", "business_id", "stars")
header = train_rdd.first() 
train_rdd = train_rdd.rdd.filter(lambda l: l!=header).map(lambda l: ((l[0],l[1]),(l[2],1))).persist()

In [26]:
train_rdd = train_rdd.reduceByKey(lambda x,y:(x[0] + y[0],x[1] + y[1])).map(lambda l: (l[0],l[1][0]/l[1][1])).sortByKey(True).persist()

In [27]:
train_rdd.take(10)

[(('---1lKK3aKOuomHnwAkAow', '-ErwgUmZ1-jHW_rSu55jlg'), 5.0),
 (('---1lKK3aKOuomHnwAkAow', '1JgaRBX0oiRsvEhHF3ZMjw'), 1.0),
 (('---1lKK3aKOuomHnwAkAow', '2BbFeotL85cIaBjSq1SWiA'), 1.0),
 (('---1lKK3aKOuomHnwAkAow', '5cbsjFtrntUAeUx51FaFTg'), 1.0),
 (('---1lKK3aKOuomHnwAkAow', '5rxJpTkeJa5rxMvL2NbSnQ'), 5.0),
 (('---1lKK3aKOuomHnwAkAow', '78TC3sZSYBzBsSJ0z5pyhw'), 1.0),
 (('---1lKK3aKOuomHnwAkAow', 'Gaasy9YbPGVc8KcXcAIqEw'), 5.0),
 (('---1lKK3aKOuomHnwAkAow', 'HZdtHOEaKUL2SlWj5owgCA'), 5.0),
 (('---1lKK3aKOuomHnwAkAow', 'OnnvNs2JJ-B2xbwHEHbWDw'), 5.0),
 (('---1lKK3aKOuomHnwAkAow', 'PT9BRvRWx9kpcu02VouW1g'), 2.0)]

In [28]:
train_collect = train_rdd.collect()
outfile = open("train_data.txt","w")
for item in train_collect:
    outfile.write(item[0][0] + ", " + item[0][1] + ", " + str(item[1]) + "\n")
outfile.close()

In [7]:
test_data = sqlContext.read.json("reviews_testing_set.json")

In [14]:
test_rdd = test_data.select("user_id", "business_id", "stars")
test_rdd = test_rdd.rdd.map(lambda l:((l[0],l[1]),l[2])).sortByKey(True).persist()
test_rdd.take(10)

[(('---1lKK3aKOuomHnwAkAow', 'Qy_tDaVTWlS14fEglzo1Tg'), 4),
 (('---1lKK3aKOuomHnwAkAow', 'bzPrrbDHRxstf9FSPw1Xjw'), 5),
 (('---1lKK3aKOuomHnwAkAow', 'hk4NPN6W2xt30Cp-Fiqtuw'), 4),
 (('---1lKK3aKOuomHnwAkAow', 'rq5dgoksPHkJwJNQKlGQ7w'), 5),
 (('---1lKK3aKOuomHnwAkAow', 'ttH3ZbUcncBRIXqT-YVPCg'), 5),
 (('--CIuK7sUpaNzalLAlHJKA', 'sk0stgY4NDJYOX1MbNJ3Pg'), 2),
 (('--EMqnd727rtC0G5Oc-Mrg', 'b_XIKJ2nNzksuWhfMTEehQ'), 5),
 (('--FjmST55XwJ710qYlUTxA', 'pkXJEEaWcljXgAUd7cGzIA'), 1),
 (('--HCoE1ghaAlcaAfshICgw', 'caUPXiOxvLGMsorJBtNlhg'), 5),
 (('--IJ1MaFBp4bLFJ1xkofPw', 'ahSFUPojs9X3-1jP-QPb-w'), 5)]

In [16]:
test_collect = test_rdd.collect()
outfile = open("test_data.txt","w")
for item in test_collect:
    outfile.write(item[0][0] + ", " + item[0][1] + ", " + str(item[1]) + "\n")
outfile.close()